In [2]:
import pandas as pd

In [2]:
def get_raw_data():
    train = pd.read_csv("train_1.csv")
    test = pd.read_csv("key_1.csv")
    return train, test

In [3]:
def transform_data(train, test, periods=-49):
    train_flattened = pd.melt(train[list(train.columns[periods:])+['Page']], id_vars='Page', var_name='date', value_name='Visits')
    train_flattened = get_features(train_flattened)
    test['date'] = test.Page.apply(lambda a: a[-10:])
    test['Page'] = test.Page.apply(lambda a: a[:-11])
    test = get_features(test)
    return train_flattened, test

In [5]:
## seems every2_weekdays not used here, just filter it out here!

In [6]:
# https://www.kaggle.com/chechir/weekend-flag-median-with-wiggle/code

In [7]:
def get_features(df):
    df['date'] = df['date'].astype('datetime64[ns]')
#     df['every2_weekdays'] = df.date.dt.dayofweek
    df['weekend'] = (df.date.dt.dayofweek // 5).astype(float)
    #df['shortweek'] = ((df.date.dt.dayofweek) // 4 == 1).astype(float)
    return df

In [8]:
def predict_using_median_weekend(train, test):
    df = train.copy()
#     df = df.drop(['every2_weekdays'], axis=1)
    agg_train_weekend = df.groupby(['Page', 'weekend']).median().reset_index()
    test_df = test.merge(agg_train_weekend, how='left')
    result = test_df['Visits'].values
    return result

In [9]:
def wiggle_preds(df):
    second_term_ixs = df['date'] < '2017-02-01'
    adjusted = df['Visits'].values + df['Visits'].values*0.02
    adjusted[second_term_ixs] = df['Visits'].values[second_term_ixs] + df['Visits'].values[second_term_ixs]*0.04
    df['Visits'] = adjusted
    df.loc[df.Visits.isnull(), 'Visits'] = 0
    return df

In [10]:
train, test = get_raw_data()
train, test = transform_data(train, test, periods=-49)
preds_weekend = predict_using_median_weekend(train, test)
train, test = get_raw_data()
train, test = transform_data(train, test, periods=-12)
preds_otherdays = predict_using_median_weekend(train, test)

weight = 0.9
preds_ensemble = preds_weekend*weight + preds_otherdays*(1-weight)
test['Visits'] = preds_ensemble
test = wiggle_preds(test)

test[['Id','Visits']].to_csv('sub_mads_weight_{}.csv'.format(weight), index=False)
print(test[['Id', 'Visits']].head(10))
print(test[['Id', 'Visits']].tail(10))

             Id  Visits
0  bf4edcf969af    3.12
1  929ed2bf52b9    2.08
2  ff29d0f51d5c    2.08
3  e98873359be6    2.08
4  fa012434263a    2.08
5  48f1e93517a2    2.08
6  5def418fcb36    3.12
7  77bd08134351    3.12
8  5889e6dbb16f    2.08
9  5f21fef1d764    2.08
                   Id   Visits
8703770  a4456a9d271d  247.554
8703771  d43a25cf4ef2  247.554
8703772  8f47d2e020cd  247.554
8703773  a78af728d84b  247.554
8703774  d1ba45c7ec08  247.554
8703775  f69747f5ee68  270.045
8703776  2489963dc503  270.045
8703777  b0624c909f4c  247.554
8703778  24a1dfb06c10  247.554
8703779  add681d54216  247.554


In [41]:
train, test = get_raw_data()
train, test = transform_data(train, test, periods=-49)

In [5]:
train.shape, test.shape

((145063, 551), (8703780, 2))

In [23]:
train.shape, test.shape

((7108087, 5), (8703780, 5))

In [24]:
train.head()

,Page,date,Visits,every2_weekdays,weekend
0,2NE1_zh.wikipedia.org_all-access_spider,2016-11-13,8.0,6,1.0
1,2PM_zh.wikipedia.org_all-access_spider,2016-11-13,11.0,6,1.0
2,3C_zh.wikipedia.org_all-access_spider,2016-11-13,4.0,6,1.0
3,4minute_zh.wikipedia.org_all-access_spider,2016-11-13,13.0,6,1.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2016-11-13,11.0,6,1.0


In [28]:
test.tail()

,Page,Id,date,every2_weekdays,weekend
8703775,龙生九子_zh.wikipedia.org_mobile-web_all-agents,f69747f5ee68,2017-02-25,5,1.0
8703776,龙生九子_zh.wikipedia.org_mobile-web_all-agents,2489963dc503,2017-02-26,6,1.0
8703777,龙生九子_zh.wikipedia.org_mobile-web_all-agents,b0624c909f4c,2017-02-27,0,0.0
8703778,龙生九子_zh.wikipedia.org_mobile-web_all-agents,24a1dfb06c10,2017-02-28,1,0.0
8703779,龙生九子_zh.wikipedia.org_mobile-web_all-agents,add681d54216,2017-03-01,2,0.0


In [9]:
train_flattened = pd.melt(train[list(train.columns[-28:])+['Page']], id_vars='Page', var_name='date', value_name='Visits')


In [1]:
len(train)

NameError: name 'train' is not defined